The postings table has data that requires flattening. Some members switched parties and these changes need to be considered a separate posting for the durations served with each party.

In [1]:
# Import Dependencies
import pandas as pd
from datetime import datetime
import csv
import re



In [2]:
# Load the postings CSV file into a DataFrame
postings_df = pd.read_csv('data\\postings_2.csv', dtype=str)
postings_df.head()

,bioguide_id,chamber,job_type,congress_number,congress_start_date,congress_end_date,region_type,region_code,party_name,job_start_date,job_end_date,party_start_date,party_end_date
0,A000002,Representative,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,VA,Democrat,NaN,NaN,NaN,NaN
1,A000016,Representative,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,MS,Democrat,NaN,NaN,NaN,NaN
2,A000024,Representative,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,IN,Republican,NaN,NaN,NaN,NaN
3,A000054,Representative,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,NJ,Democrat,NaN,NaN,NaN,NaN
4,A000062,Senator,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,VT,Republican,NaN,NaN,NaN,NaN


In [3]:
postings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36714 entries, 0 to 36713
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   bioguide_id          36714 non-null  object
 1   chamber              36714 non-null  object
 2   job_type             36714 non-null  object
 3   congress_number      36714 non-null  object
 4   congress_start_date  36714 non-null  object
 5   congress_end_date    36714 non-null  object
 6   region_type          36598 non-null  object
 7   region_code          36598 non-null  object
 8   party_name           36694 non-null  object
 9   job_start_date       1662 non-null   object
 10  job_end_date         1444 non-null   object
 11  party_start_date     80 non-null     object
 12  party_end_date       62 non-null     object
dtypes: object(13)
memory usage: 3.6+ MB


In [4]:
postings_df.shape

(36714, 13)

In [5]:
# Remove duplicate rows from postings_df
duplicates = postings_df[postings_df.duplicated()]
print(f"Number of duplicate rows: {len(duplicates)}")
duplicates.shape


Number of duplicate rows: 18357


(18357, 13)

In [ ]:
# # Convert date columns to date format
# date_columns = ['congress_start_date', 'congress_end_date', 'job_start_date', 'job_end_date']
# for col in date_columns:
#     postings_df[col] = pd.to_datetime(postings_df[col], errors='coerce')
# postings_df.info()


postings_2 extracted data from mongoDB 

In [9]:
# Make a list of bioguide_ids where the party_name column contains a list containing more than one party
# Ignore NaN values
multi_party_bioguide_ids = postings_df[postings_df['party_name'].str.contains(',', na=False)]['bioguide_id'].unique().tolist()
# print the list
print(multi_party_bioguide_ids)
# print a count of how many bioguide_ids are in the list
print(f"Number of bioguide_ids with multiple parties: {len(multi_party_bioguide_ids)}")

[]
Number of bioguide_ids with multiple parties: 0


In [ ]:
# Make a dataframe for bioguide_id A000367 (Example case for multiple parties)
A000367_df = postings_df[postings_df['bioguide_id'] == 'A000367']
A000367_df.head(20)


,bioguide_id,chamber,job_type,congress_number,congress_start_date,congress_end_date,region_type,region_code,party_name,job_start_date,job_end_date,party_start_date,party_end_date
28900,A000367,Representative,CongressMemberJob,112,2011-01-03,2013-01-03,StateRegion,MI,Republican,NaN,NaN,NaN,NaN
30008,A000367,Representative,CongressMemberJob,113,2013-01-03,2015-01-03,StateRegion,MI,Republican,NaN,NaN,NaN,NaN
31122,A000367,Representative,CongressMemberJob,114,2015-01-03,2017-01-03,StateRegion,MI,Republican,NaN,NaN,NaN,NaN
32224,A000367,Representative,CongressMemberJob,115,2017-01-03,2019-01-03,StateRegion,MI,Republican,NaN,NaN,NaN,NaN
33354,A000367,Representative,CongressMemberJob,116,2019-01-03,2021-01-03,StateRegion,MI,Republican,NaN,NaN,2019-01-03,2019-07-04
33355,A000367,Representative,CongressMemberJob,116,2019-01-03,2021-01-03,StateRegion,MI,Independent,NaN,NaN,2019-07-04,2020-05-01
33356,A000367,Representative,CongressMemberJob,116,2019-01-03,2021-01-03,StateRegion,MI,Libertarian,NaN,NaN,2020-05-01,NaN


A data error was found during exploration. The start date of the first congress served would be found in the start date of the final term served.

In [27]:
# Make a list of bioguide_ids where the job_start_date is earlier than the congress_start_date
date_error_bioguide_ids = postings_df[pd.to_datetime(postings_df['job_start_date'], errors='coerce') < pd.to_datetime(postings_df['congress_start_date'], errors='coerce')]['bioguide_id'].unique().tolist()
print(date_error_bioguide_ids)
# print a count of how many bioguide_ids are in the list
print(f"Number of bioguide_ids with job_start_date earlier than congress_start_date: {len(date_error_bioguide_ids)}")


['B000223', 'P000152', 'B001251', 'B001270', 'D000482', 'G000582', 'M001165']
Number of bioguide_ids with job_start_date earlier than congress_start_date: 7


In [ ]:
# Clean the postings data, correct for date errors
# updated posting start and end dates based on the following rules:
# 1. The posting start date cannot be after the posting end date.
# 2. The posting start date cannot be before the session start date.
# 3. The posting end date cannot be after the session end date.